数据介绍：
本次作業使用豐原站的觀測記錄，分成train set跟test set，train set是豐原站每個月的前20天所有資料test set則是從豐原站剩下的資料中取樣出來。
train.csv:每個月前20天每個小時的氣象資料(每小時有18種測資)。共12個月。
test.csv:從剩下的資料當中取樣出連續的10小時為一筆，前九小時的所有觀測數據當作feature，第十小時的PM2.5當作answer。
一共取出240筆不重複的test data，請根據feauure預測這240筆的PM2.5。


In [262]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [23]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test(1).csv")

In [24]:
train.head()

,Date,stations,observation,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,station,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,station,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,station,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,station,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,station,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5


In [25]:
test.head()

,id_0,AMB_TEMP,15,14,14.1,13,13.1,13.2,13.3,13.4,12
0,id_0,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
1,id_0,CO,0.36,0.35,0.34,0.33,0.33,0.34,0.34,0.37,0.42
2,id_0,NMHC,0.11,0.09,0.09,0.1,0.1,0.1,0.1,0.11,0.12
3,id_0,NO,0.6,0.4,0.3,0.3,0.3,0.7,0.8,0.8,0.9
4,id_0,NO2,9.3,7.1,6.1,5.7,5.5,5.3,5.5,7.1,7.5


In [26]:
train = train[train['observation'] == 'PM2.5']
test = test[test['AMB_TEMP'] == 'PM2.5']

In [27]:
train.head()

,Date,stations,observation,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
9,2014/1/1,station,PM2.5,26,39,36,35,31,28,25,...,36,45,42,49,45,44,41,30,24,13
27,2014/1/2,station,PM2.5,21,23,30,30,22,18,13,...,53,43,43,45,46,32,16,19,22,26
45,2014/1/3,station,PM2.5,19,25,27,20,16,14,15,...,32,36,34,45,40,41,23,29,23,37
63,2014/1/4,station,PM2.5,27,27,14,20,22,24,26,...,62,55,56,67,78,83,90,75,85,82
81,2014/1/5,station,PM2.5,80,80,76,81,75,66,70,...,64,73,57,57,53,70,70,60,68,66


In [28]:
test.head()

,id_0,AMB_TEMP,15,14,14.1,13,13.1,13.2,13.3,13.4,12
8,id_0,PM2.5,27,13,24,29,41,30,29,27,28
26,id_1,PM2.5,46,47,57,78,84,76,59,61,61
44,id_2,PM2.5,10,10,25,34,40,39,36,25,22
62,id_3,PM2.5,71,58,51,41,41,46,43,34,29
80,id_4,PM2.5,13,23,18,10,5,5,13,9,12


In [29]:
train = train.iloc[:,3:]
test = test.iloc[:,2:]

In [30]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
9,26,39,36,35,31,28,25,20,19,30,...,36,45,42,49,45,44,41,30,24,13
27,21,23,30,30,22,18,13,13,11,22,...,53,43,43,45,46,32,16,19,22,26
45,19,25,27,20,16,14,15,8,4,9,...,32,36,34,45,40,41,23,29,23,37
63,27,27,14,20,22,24,26,33,48,50,...,62,55,56,67,78,83,90,75,85,82
81,80,80,76,81,75,66,70,65,66,57,...,64,73,57,57,53,70,70,60,68,66


In [51]:
"""
此处的作用是扩大训练集以及与测试集对应起来
从第1列数据连续9列数据作为x，第10例作为y，然后从第二列数据开始
一共有24列数组，因此得到15组数据，X的每组数据是240*9，Y是240*1
"""
train_x = []
train_y = []
for i in range(15):
    x = train.iloc[:, i:i + 9]
    x.columns = np.array(range(9))  
    y = train.iloc[:, i + 9]
    y.columns = np.array(range(1))
    train_x.append(x)
    train_y.append(y)

In [52]:
# 将train_x中的15组数据拼接成一组数据
train_x = pd.concat(train_x).T
train_y = pd.concat(train_y).T


In [70]:
#将str数据变成float
train_x = np.array(train_x,float)
train_y = np.array(train_y,float)

In [71]:
print(train_x.shape)
print(train_y.shape)

(9, 3600)
(3600,)


In [72]:
train_y = train_y.reshape(1,train_x.shape[1])

In [195]:
print(train_y.shape)

(1, 3600)


In [237]:
#初始化参数 w随机初始化，防止鞍点
np.random.seed(3)
W = np.random.randn(1,train_x.shape[0]) * 0.01
b = np.zeros((1,1))
W

array([[ 0.01788628,  0.0043651 ,  0.00096497, -0.01863493, -0.00277388,
        -0.00354759, -0.00082741, -0.00627001, -0.00043818]])

In [281]:
def computer_cost(x, y, w,b):
    return 1/(2*x.shape[1])*np.sum((np.dot(w,x)+ b - y)**2)

In [282]:
computer_cost(train_x,train_y,W,b)

438.27375974240215

In [283]:
#梯度下降
def grad_desc(x,y,W,b,iteration,learning_rate):
    m = x.shape[1] #样本总数
    costs = []
    for i in np.arange(iteration):
        Z = np.dot(W,x) + b
        dW = 1/ m * np.sum(np.dot(x,(Z - y).T))
        db = 1/ m * np.sum(Z - y)
        W = W - learning_rate * dW
        b = b - learning_rate * db
        if i % 100 == 0:
            cost = computer_cost(x,y,W,b) 
            print("Cost after iteration {}: {}".format(i, cost))
            costs.append(cost)
    return W,b,costs
    

In [284]:
# 数据归一化，若不归一化，数据收敛特别慢
ss = StandardScaler()
ss.fit(train_x)
train_x = ss.transform(train_x)
# ss.fit(test_x)
# test_x = ss.transform(test_x)

In [303]:
W_,b_,costs = grad_desc(train_x,train_y,W,b,2000,0.5)

Cost after iteration 0: 221.15296319652077
Cost after iteration 100: 148.7793643478937
Cost after iteration 200: 148.7793643478937
Cost after iteration 300: 148.7793643478937
Cost after iteration 400: 148.7793643478937
Cost after iteration 500: 148.7793643478937
Cost after iteration 600: 148.7793643478937
Cost after iteration 700: 148.7793643478937
Cost after iteration 800: 148.7793643478937
Cost after iteration 900: 148.7793643478937
Cost after iteration 1000: 148.7793643478937
Cost after iteration 1100: 148.7793643478937
Cost after iteration 1200: 148.7793643478937
Cost after iteration 1300: 148.7793643478937
Cost after iteration 1400: 148.7793643478937
Cost after iteration 1500: 148.7793643478937
Cost after iteration 1600: 148.7793643478937
Cost after iteration 1700: 148.7793643478937
Cost after iteration 1800: 148.7793643478937
Cost after iteration 1900: 148.7793643478937


In [302]:
W_

array([[ 0.01788628,  0.0043651 ,  0.00096497, -0.01863493, -0.00277388,
        -0.00354759, -0.00082741, -0.00627001, -0.00043818]])

In [296]:
test_x = pd.read_csv('test(1).csv')
test_x = test_x[test_x['AMB_TEMP']=='PM2.5']
test_x = test_x.iloc[:,2:].T
test_x=np.array(test_x,float)

In [297]:
test_x

array([[27., 46., 10., ..., 16., 29., 12.],
       [13., 47., 10., ..., 25., 27.,  8.],
       [24., 57., 25., ..., 12., 23.,  7.],
       ...,
       [29., 59., 36., ..., 11., 35.,  0.],
       [27., 61., 25., ...,  3., 35.,  0.],
       [28., 61., 22., ...,  0., 26.,  0.]])

In [300]:
y_pre = np.dot(W_,test_x) + b_
y_pre


array([[23.65889719, 22.73095396, 23.22974816, 24.33208487, 24.12207311,
        23.58650895, 22.94565129, 24.04375212, 24.03390583, 23.75884705,
        23.69640265, 23.23999702, 23.79459183, 24.25881143, 23.40711553,
        24.10758683, 23.16801167, 24.32132815, 24.09326119, 24.40712331,
        24.05254185, 24.39916309, 24.09104178, 24.44187746, 23.38732727,
        23.95914782, 23.77528559, 23.62956835, 23.55070925, 24.091186  ,
        24.29872388, 24.2985625 , 23.61381975, 25.2183365 , 24.11648665,
        23.13259333, 23.27309151, 23.28722866, 23.98259205, 23.40463888,
        23.31854791, 23.60626619, 23.31441255, 23.96175152, 23.73590423,
        23.07050561, 23.45965836, 23.4573234 , 23.51854176, 24.09989996,
        23.62225148, 24.17177221, 23.37322272, 24.07295148, 23.0508966 ,
        23.19220602, 24.16064502, 23.98588901, 23.1773772 , 23.39011926,
        24.16415606, 22.57647968, 23.7699897 , 23.50638363, 24.09257099,
        24.04004671, 24.30979829, 22.66472989, 23.7